In [1]:
!pip install --upgrade plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 48.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
!unzip 2021-olympics-in-tokyo.zip

Dataset URL: https://www.kaggle.com/datasets/arjunprasadsarkhel/2021-olympics-in-tokyo
License(s): CC-BY-SA-4.0
  0% 0.00/348k [00:00<?, ?B/s]
100% 348k/348k [00:00<00:00, 88.6MB/s]
Archive:  2021-olympics-in-tokyo.zip
  inflating: Athletes.xlsx           
  inflating: Coaches.xlsx            
  inflating: EntriesGender.xlsx      
  inflating: Medals.xlsx             
  inflating: Teams.xlsx              


In [3]:
!pip install pyspark


In [4]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())

spark

In [6]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv',sep=',',index=False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')




/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
gender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

In [11]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [12]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [16]:
medals.sort_values(by='Rank by Total',inplace = True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [19]:
import plotly.express as px
fig = px.bar(medals, x='Team/NOC', y =['Gold','Silver','Bronze'],color_discrete_sequence=['Gold','silver','brown']
             ,title='No. of Medals by Country')
fig.show()

In [20]:
coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [21]:
spark.read.csv('Coaches.csv',header=True,inferSchema=True).createOrReplaceTempView('coaches')
spark.sql('select * from coaches').show()

+--------------------+--------------------+-----------------+--------+
|                Name|                 NOC|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [26]:
pd_coaches_query = spark.sql('select NOC, COUNT(NOC) as count from coaches GROUP BY NOC ORDER BY NOC').toPandas()
pd_coaches_query.head(7)

,NOC,count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [28]:
fig = px.bar(pd_coaches_query, x='NOC',y='count',color='count',title='No. of Coaches from Each Country')
fig.show()

In [29]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [32]:
spark.read.csv('Athletes.csv',header=True,inferSchema=True).createOrReplaceTempView('athlete')
spark.sql('select * from athlete order by NOC, Discipline').show()

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
| LAHOULOU Abdelmalik|    Algeria|          Athletics|
|TRIKI Yas

In [37]:
spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
          count(*) as count from athlete group by NOC, Discipline
          order by NOC, Discipline ''').show()

+-----------+-------------------+-----+
|        NOC|         Discipline|count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [38]:
pd_athlete_query=spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
          count(*) as count from athlete group by NOC, Discipline
          order by NOC, Discipline ''').toPandas()

In [42]:
fig = px.bar(pd_athlete_query,x='NOC',y='count',color='Discipline',title='No. of Athlete by country')

fig.show()

In [44]:
athletes['Discipline'].value_counts().head(20)

,count
Discipline,
Athletics,2068
Swimming,743
Football,567
Rowing,496
Hockey,406
Judo,373
Handball,343
Shooting,342
Sailing,336


In [46]:
pd_athlete_query1 = pd_athlete_query.loc[pd_athlete_query['Discipline'].isin(['Athletics','Swimming','Football'])]
pd_athlete_query1.Discipline.value_counts()

,count
Discipline,
Athletics,195
Swimming,181
Football,24


In [47]:
fig = px.bar(pd_athlete_query1,x='NOC',y='count',color='Discipline',title='No. of Athlete by country')

fig.show()

In [48]:
gender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [49]:
len(gender)

46

In [59]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'},{'type':'domain'}]] * 23
fig = make_subplots(rows=23,cols=2,subplot_titles=gender['Discipline'],specs=specs)
sub_figs = []

for index,row in gender.iterrows():
  sub_fig = go.Pie(labels=['Female','Male'],values=[row['Female'],row['Male']])
  sub_figs.append(sub_fig)


k = 0
for i in range(1,24):
  for j in range(1,3):
    fig.add_trace(sub_figs[k],i,j)
    k+=1


fig.update_layout(showlegend=False,height=10000,width=800,title_text='Gender Distribution')
fig.update_traces(textposition='inside',textinfo='percent+label',hoverinfo='label+percent+value')

fig = go.Figure(fig)
fig.show()

In [60]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [61]:
spark.read.csv('Teams.csv',header=True,inferSchema=True).createOrReplaceTempView('teams')
spark.sql('select * from teams').show()

+-------------+--------------+--------------------+------------+
|         Name|    Discipline|                 NOC|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [66]:
team_pd_query = spark.sql('''select NOC, Discipline , COUNT(*) as counts from teams group by NOC,Discipline
          order by NOC,Discipline ''').toPandas()
team_pd_query.head(7)

,NOC,Discipline,counts
0,Angola,Handball,1
1,Argentina,Basketball,1
2,Argentina,Beach Volleyball,2
3,Argentina,Football,1
4,Argentina,Handball,1
5,Argentina,Hockey,2
6,Argentina,Rugby Sevens,1


In [69]:
fig = px.bar(team_pd_query,x='NOC',y='counts',color='Discipline',title='No. of Teams by Country')
fig.show()

In [71]:
fig = px.scatter(team_pd_query,x='NOC',y='counts',color='Discipline',title='No. of Teams by Country')
fig.show()